# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lats_lngs = []
cities = []
geocordinates = []
country_codes = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lats_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lats_lngs:
    
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    code = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        geocordinates.append(lat_lng)
        country_codes.append(code)

# Print the city count to confirm sufficient amount of cities
print(f"City count: {len(cities)} and geocoordinate count: {len(geocordinates)}.")

City count: 615 and geocoordinate count: 615.


In [3]:
# Create a data frame to collect cities, country codes, and coordinates
city_geo_df = pd.DataFrame({'Cities': cities, 
                            'Country Code': country_codes, 
                            'Geo_Coord.': geocordinates,})
# Check city_geo_df 
city_geo_df.head()

,Cities,Country Code,Geo_Coord.
0,mataura,pf,"(-34.64127820557524, -143.89083372137654)"
1,punta arenas,cl,"(-73.3028158255195, -95.24947304510074)"
2,ketchikan,us,"(55.64245959762383, -130.73425502560104)"
3,pacific grove,us,"(27.14963895997208, -133.03958141548225)"
4,puerto ayora,ec,"(-13.153568496735446, -100.33898177448928)"


In [4]:
# Create a column to combine City and Country
# Will be used in OpenWeatherMap API
city_geo_df['City,Country'] = city_geo_df['Cities'] + ',' + city_geo_df['Country Code']

# Check city_geo_df and new column, City,Country
city_geo_df.head()

,Cities,Country Code,Geo_Coord.,"City,Country"
0,mataura,pf,"(-34.64127820557524, -143.89083372137654)","mataura,pf"
1,punta arenas,cl,"(-73.3028158255195, -95.24947304510074)","punta arenas,cl"
2,ketchikan,us,"(55.64245959762383, -130.73425502560104)","ketchikan,us"
3,pacific grove,us,"(27.14963895997208, -133.03958141548225)","pacific grove,us"
4,puerto ayora,ec,"(-13.153568496735446, -100.33898177448928)","puerto ayora,ec"


In [5]:
# Create a new DataFrame to prepare to include city,country, 
#  latitudes, and weather information
city_weather_df = pd.DataFrame({'City,Country':city_geo_df['City,Country']})
city_weather_df['Latitude'] = ""
city_weather_df['Longitude'] = ""
city_weather_df['Temperature'] = ""
city_weather_df['Humidity'] = ""
city_weather_df['Cloudiness'] = ""
city_weather_df['Wind Speed (mph)'] = ""

# Check new city_weather_df 
city_weather_df.head()

,"City,Country",Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed (mph)
0,"mataura,pf",,,,,,
1,"punta arenas,cl",,,,,,
2,"ketchikan,us",,,,,,
3,"pacific grove,us",,,,,,
4,"puerto ayora,ec",,,,,,


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [6]:
# Test calling Current Weather Data API from OpenWeatherMap

# Test base url to use
test_url = "http://api.openweathermap.org/data/2.5/weather?"

# Test city
test_city = "London,UK"

# Parameter to make temperature read in Fahrenheit
test_temp_unit = "imperial"

# Test target url using test_city, api_key, test_temp_unit
target_url = test_url + "q={0}&APPID={1}&units={2}".format(test_city, api_key, test_temp_unit)

# Request for test weather infromation for London, UK and format in json
test_results = requests.get(target_url).json()

# Print test_results of London weather in json format
test_results

{'coord': {'lon': -0.13, 'lat': 51.51},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 44.8,
  'pressure': 1037,
  'humidity': 81,
  'temp_min': 42.8,
  'temp_max': 46.4},
 'visibility': 10000,
 'wind': {'speed': 3.36, 'deg': 260},
 'clouds': {'all': 90},
 'dt': 1546818600,
 'sys': {'type': 1,
  'id': 1417,
  'message': 0.0066,
  'country': 'GB',
  'sunrise': 1546848282,
  'sunset': 1546877336},
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [7]:
# Test extracting infromation from test_results
# Looking at main temperature and cloudiness 
test_temp = test_results['main']['temp']
test_cloudiness = test_results['clouds']['all']
test_lat = test_results['coord']['lat']

print(f"Test results for London,UK: main temp - {test_temp},")
print(f"cloudiness - {test_cloudiness}, and lat is {test_lat}.")

Test results for London,UK: main temp - 44.8,
cloudiness - 90, and lat is 51.51.


In [8]:
# Loop through the city_weater_df and get weather data

# Set up params for OpenWeatherAPI
params = {"APPID": api_key,}

# Use iterrows to iterate through city_weather_df
for index, row in city_weather_df.iterrows():
   
    # Base url for OpenWeatherMap API
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    
    # Get city,country info from city_weather_df:
    city_country = row["City,Country"]
    
    
    # Add Keyword to parameters dictionary:
    params['q'] = city_country
    params['units'] = "imperial"
    
    # Request to endpoint and convert result to json
    city_results = requests.get(base_url, params=params).json()

    try:
        city_weather_df.loc[index, 'Temperature'] = city_results['main']['temp']
        city_weather_df.loc[index, 'Humidity'] = city_results['main']['humidity']
        city_weather_df.loc[index, 'Cloudiness'] = city_results['clouds']['all']
        city_weather_df.loc[index, 'Wind Speed (mph)'] = city_results['wind']['speed']
        city_weather_df.loc[index, 'Latitude'] = city_results['coord']['lat']
        city_weather_df.loc[index, 'Longitude'] = city_results['coord']['lon']
        
    except (KeyError, IndexError):
        print(f"{city_country} cannot be found or has missing data")

city_weather_df.head()

mataura,pf cannot be found or has missing data
attawapiskat,ca cannot be found or has missing data
richard toll,sn cannot be found or has missing data
safaga,eg cannot be found or has missing data
airai,pw cannot be found or has missing data
nizhneyansk,ru cannot be found or has missing data
burica,pa cannot be found or has missing data
marcona,pe cannot be found or has missing data
mys shmidta,ru cannot be found or has missing data
saleaula,ws cannot be found or has missing data
mme,cm cannot be found or has missing data
illoqqortoormiut,gl cannot be found or has missing data
gat,ly cannot be found or has missing data
tumannyy,ru cannot be found or has missing data
maunabo,us cannot be found or has missing data
gorkovskoye,ru cannot be found or has missing data
belushya guba,ru cannot be found or has missing data
olafsvik,is cannot be found or has missing data
kralendijk,an cannot be found or has missing data
barentsburg,sj cannot be found or has missing data
japura,br cannot be found

,"City,Country",Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed (mph)
0,"mataura,pf",,,,,,
1,"punta arenas,cl",-53.16,-70.91,46.4,95,40,14.99
2,"ketchikan,us",55.34,-131.65,30.04,48,1,16.11
3,"pacific grove,us",36.62,-121.92,51.91,77,90,17.22
4,"puerto ayora,ec",-0.74,-90.35,82.4,65,20,16.11


In [9]:
# Confirm count of cities in city_weather_df 
print(f" The city_weather_df has {len(city_weather_df)} cities.")


 The city_weather_df has 615 cities.


In [10]:
# Drop cities from city_weather_df that were not found in OpenWeatherMap data

# First, replace empty values in DataFram with NaN
city_weather_df.replace('', np.nan, inplace=True)

# Check that empty values were replaced with NaN
city_weather_df.head()

,"City,Country",Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed (mph)
0,"mataura,pf",NaN,NaN,NaN,NaN,NaN,NaN
1,"punta arenas,cl",-53.16,-70.91,46.40,95.0,40.0,14.99
2,"ketchikan,us",55.34,-131.65,30.04,48.0,1.0,16.11
3,"pacific grove,us",36.62,-121.92,51.91,77.0,90.0,17.22
4,"puerto ayora,ec",-0.74,-90.35,82.40,65.0,20.0,16.11


In [11]:
# Drop row that have NaN values
city_weather_df.dropna(inplace=True)

# Check that drops occured
city_weather_df.head()

,"City,Country",Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed (mph)
1,"punta arenas,cl",-53.16,-70.91,46.40,95.0,40.0,14.99
2,"ketchikan,us",55.34,-131.65,30.04,48.0,1.0,16.11
3,"pacific grove,us",36.62,-121.92,51.91,77.0,90.0,17.22
4,"puerto ayora,ec",-0.74,-90.35,82.40,65.0,20.0,16.11
5,"cape town,za",-33.93,18.42,64.40,82.0,0.0,8.05


In [12]:
# Print the new number of cities that have complete weather data
print(f"city_weather_df has {len(city_weather_df)} cities with complete weather data.")

city_weather_df has 538 cities with complete weather data.


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [14]:
# Create out put csv file of data
output_data_file = "OpenWeatherMap_Data_538_Cities.csv"

city_weather_df.to_csv(output_data_file)

In [ ]:
city_weather_df.to_html()